# Import Canada Borough Data

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')

In [4]:
#print(soup.prettify())

## Pull Out Table

In [5]:
tables = soup.find_all('table', class_='sortable')

### Extract Data by Heading

In [6]:
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postcode', 'Borough', 'Neighbourhood']:
        break

In [7]:
#print(table)

### Put Data in Dataframe

In [8]:
P = []
B = []
N = []
with open('CanadaArea.txt', 'w') as fo:
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        Postcode, Borough, Neighbourhood = [td.text.strip() for td in tds[:3]]
        P.append(Postcode)
        B.append(Borough)
        N.append(Neighbourhood)

In [9]:
d = pd.DataFrame({'Postcode':P,'Borough':B,'Neighbourhood':N})
d.head()
d.shape

(288, 3)

### Remove Not Assigned Stuff

In [10]:
drop = []
for dems in range(len(d)):
    if d['Borough'].iloc[dems] == 'Not assigned':
        drop.append(dems)

In [11]:
df = d.drop(d.index[drop])
df.shape

(211, 3)

In [12]:
df.head()


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Combine similar postcodes

In [13]:
drop2 = []
for dems in range(len(df)-1):
    if df['Postcode'].iloc[dems] == df['Postcode'].iloc[dems+1]:
        df['Neighbourhood'].iloc[dems] =  df['Neighbourhood'].iloc[dems] + ', ' + df['Neighbourhood'].iloc[dems+1]
        drop2.append(dems+1)

In [14]:
df2 = df.drop(df.index[drop2])

### Replace the one and only 'not assigned' neighborhood with Borough

In [15]:
for i in range(len(df2)):
    if df2['Neighbourhood'].iloc[i] == 'Not assigned':
        df2['Neighbourhood'].iloc[i] = df2['Borough'].iloc[i]

In [16]:
df2.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"
8,M7A,Queen's Park,Queen's Park


### Reset index so it looks nice

In [17]:
df2 = df2.reset_index(drop=True)

In [18]:
df2.shape

(103, 3)

### GEO from google doesn't work, so using excel doc

In [19]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
latitude = []
longitude = []

for data in range(2):
    postal_code=df2.iloc[data]

    # loop until you get the coordinates
    #while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    
#add lat/lon to array
#add array to dataframe

TypeError: 'NoneType' object is not subscriptable

In [20]:
import csv
geodata = []
output=[]
with open('Geospatial_Coordinates.csv', newline='') as csvfile:
    georeader = csv.reader(csvfile, delimiter=',')
    for row in georeader:
        geodata.append(row)

In [21]:
dfg = pd.DataFrame(geodata,columns={'Postal Code':0,'Latitude':0,'Longitude':0})

In [22]:
dfg.head(1)

,Postal Code,Latitude,Longitude
0,Postal Code,Latitude,Longitude


In [23]:
dfg = dfg[1:]

In [24]:
df2.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [25]:
dfg.head(2)

,Postal Code,Latitude,Longitude
1,M1B,43.8066863,-79.1943534
2,M1C,43.7845351,-79.1604971


In [26]:
df2.iloc[0]['Postcode']

'M3A'

In [27]:
dfg.iloc[0]['Postal Code']

'M1B'

### Attach Lat Lon Coords to Data

In [28]:
lat=[]
lon=[]
for row in range(len(df2)):
    for codes in range(len(dfg)):
        if df2.iloc[row]['Postcode'] == dfg.iloc[codes]['Postal Code']:
            lat.append(dfg.iloc[codes]['Latitude'])
            lon.append(dfg.iloc[codes]['Longitude'])

In [29]:
ll = pd.DataFrame({'Latitude':lat,'Longitude':lon})

In [38]:
result = pd.concat([df2, ll], axis=1, join='inner')

In [39]:
result.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.4647633
4,M7A,Queen's Park,Queen's Park,43.6623015,-79.3894938
5,M9A,Etobicoke,Islington Avenue,43.6678556,-79.5322424
6,M1B,Scarborough,"Rouge, Malvern",43.8066863,-79.1943534
7,M3B,North York,Don Mills North,43.7459058,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.7063972,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.6571618,-79.3789371


In [33]:
result.to_csv('CanadaLocData.csv')

In [15]:
import csv
import pandas as pd
result = []
output=[]
with open('CanadaLocData.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        result.append(row)
        
result = pd.DataFrame(result,columns={'Iter':0,'Postcode':0, 'Borough':0, 'Neighborhood':0,'Latitude':0,'Longitude':0})
result = result.drop('Iter', axis=1)

### Set dataset for Toronto Only

In [16]:
drop=[]
for row in range(len(result)):
    if 'Toronto' not in result.iloc[row]['Borough']:
        drop.append(row)

In [17]:
tor = result.drop(result.index[drop])

In [18]:
tor = tor.reset_index(drop=True)
tor.head(20)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6542599,-79.3606359
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.6571618,-79.3789371
2,M5C,Downtown Toronto,St. James Town,43.6514939,-79.3754179
3,M4E,East Toronto,The Beaches,43.6763574,-79.2930312
4,M5E,Downtown Toronto,Berczy Park,43.6447708,-79.3733064
5,M5G,Downtown Toronto,Central Bay Street,43.6579524,-79.3873826
6,M6G,Downtown Toronto,Christie,43.669542,-79.4225637
7,M5H,Downtown Toronto,"Adelaide, King",43.6505712,-79.3845675
8,M6H,West Toronto,"Dovercourt Village, Dufferin",43.6690051,-79.4422593
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands",43.6408157,-79.3817523


In [19]:
tor.iloc[17]['Longitude']

'-79.3887901'

### Generate Map of Neighborhoods

In [140]:
import folium
# create map of Toronto using latitude and longitude values
latitude = tor.iloc[15]['Latitude']
longitude = tor.iloc[15]['Longitude']
map_T = folium.Map(location=[float(latitude), float(longitude)], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(tor['Latitude'], tor['Longitude'], tor['Borough'], tor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_T)  
    
map_T

### From here down I am following the New York Analysis

In [141]:
import requests
CLIENT_ID = 'T4VRPSYEVOLDPDHPSDW3VC1OAU1YYDCHMFOIBULZWJRETILU' # your Foursquare ID
CLIENT_SECRET = '1JGOFHRKUQTN33EXCAUZDGQSMQHKYQU25CGF1SZSUEN354ZG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 150

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T4VRPSYEVOLDPDHPSDW3VC1OAU1YYDCHMFOIBULZWJRETILU
CLIENT_SECRET:1JGOFHRKUQTN33EXCAUZDGQSMQHKYQU25CGF1SZSUEN354ZG


In [142]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [144]:
ven = getNearbyVenues(tor['Neighborhood'], tor['Latitude'], tor['Longitude'])

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park
Deer Park, Forest Hill SE
CN Tower, Bathurst Quay
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
First Canadian Place, Underground city
Church and Wellesley
Business Reply Mail Processing Centre 969 Eastern


In [145]:
ven.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.6542599,-79.3606359,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.6542599,-79.3606359,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.6542599,-79.3606359,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.6542599,-79.3606359,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront, Regent Park",43.6542599,-79.3606359,Body Blitz Spa East,43.654735,-79.359874,Spa


In [146]:
print(ven.shape)
print('There are {} uniques categories.'.format(len(ven['Venue Category'].unique())))
ven.groupby('Neighborhood').count().head(7)

(1708, 7)
There are 240 uniques categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place",20,20,20,20,20,20
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay",13,13,13,13,13,13
"Cabbagetown, St. James Town",43,43,43,43,43,43
Central Bay Street,84,84,84,84,84,84


In [147]:
manhattan_venues = ven
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [148]:
manhattan_onehot.shape

(1708, 240)

In [149]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King",0.0,0.01,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.010000,0.0,0.0,0.01,0.0
1,Berczy Park,0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.018182,0.0,0.0,0.00,0.0
2,"Brockton, Exhibition Place",0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
4,"CN Tower, Bathurst Quay",0.0,0.00,0.0,0.0,0.076923,0.076923,0.076923,0.153846,0.153846,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0


In [150]:
manhattan_grouped.shape

(38, 240)

In [151]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King----
             venue  freq
0      Coffee Shop  0.06
1             Café  0.04
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2    Restaurant  0.04
3    Steakhouse  0.04
4          Café  0.04


----Brockton, Exhibition Place----
                    venue  freq
0                    Café  0.10
1          Breakfast Spot  0.10
2             Coffee Shop  0.10
3           Grocery Store  0.05
4  Furniture / Home Store  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1         Pizza Place  0.06
2    Recording Studio  0.06
3             Brewery  0.06
4                 Spa  0.06


----CN Tower, Bathurst Quay----
              venue  freq
0  Airport Terminal  0.15
1   Airport Service  0.15
2    Airport Lounge  0.15
3             Plane  0.08
4      Airport Gate  0.08


----Cabbagetown, St. Jam

In [152]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [153]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King",Coffee Shop,Bar,Café,Thai Restaurant,Steakhouse,Sushi Restaurant,Restaurant,Burger Joint,Bakery,American Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Café,Farmers Market,Bakery,Steakhouse,Pub,Seafood Restaurant,Cheese Shop
2,"Brockton, Exhibition Place",Breakfast Spot,Coffee Shop,Café,Gym,Furniture / Home Store,Falafel Restaurant,Bar,Stadium,Burrito Place,Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Garden,Garden Center,Auto Workshop,Recording Studio,Skate Park,Brewery,Fast Food Restaurant,Spa,Burrito Place
4,"CN Tower, Bathurst Quay",Airport Terminal,Airport Service,Airport Lounge,Airport Gate,Sculpture Garden,Plane,Harbor / Marina,Airport Food Court,Airport,Boat or Ferry


In [154]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [155]:
manhattan_data = tor
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [156]:

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6542599,-79.3606359,1,Coffee Shop,Bakery,Café,Pub,Park,Mexican Restaurant,Breakfast Spot,Theater,Dessert Shop,Shoe Store
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.6571618,-79.3789371,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Lingerie Store,Restaurant,Ramen Restaurant,Diner,Pizza Place
2,M5C,Downtown Toronto,St. James Town,43.6514939,-79.3754179,1,Coffee Shop,Restaurant,Café,Hotel,Breakfast Spot,Gastropub,Bakery,Cosmetics Shop,Italian Restaurant,Farmers Market
3,M4E,East Toronto,The Beaches,43.6763574,-79.2930312,0,Pub,Health Food Store,Coffee Shop,Wings Joint,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.6447708,-79.3733064,1,Coffee Shop,Cocktail Bar,Restaurant,Café,Farmers Market,Bakery,Steakhouse,Pub,Seafood Restaurant,Cheese Shop


In [157]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[float(latitude), float(longitude)], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lon)],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters